In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.prepare_data as pd
import utils.xyzuvd as xyzuvd
import utils.visual as visual
from utils.directory import DATA_DIR, DATASET_DIR

In [ ]:
save_prefix = 'train_fpha'
keys = pd.get_keys(save_prefix)
xyz_gt = pd.read_all_lmdb_from_name(keys, save_prefix, 'xyz_gt', 'float32', (21, 3))
uvd_gt_resize = pd.read_all_lmdb_from_name(keys, save_prefix, 'uvd_gt_resize', 'float32', (21, 3))
hand_cell_idx = pd.read_all_lmdb_from_name(keys, save_prefix, 'hand_cell_idx', 'uint8', (845))

ccs_gt = xyzuvd.xyz2ccs_color(np.array(xyz_gt))
uvd_gt = xyzuvd.xyz2uvd_color(np.array(xyz_gt))
REORDER = visual.REORDER

In [ ]:
idx = 0
file_name_i = keys[idx]
img = pd.get_img(file_name_i)
uvd_gt_i = uvd_gt[idx]
ccs_gt_i = ccs_gt[idx]
hand_cell_idx_i = hand_cell_idx[idx]
uvd_gt_resize_i = uvd_gt_resize[idx]
print(img.shape)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
visual.visualize_joints_2d(ax, uvd_gt_i[REORDER], joint_idxs=False)

In [ ]:
resize_img = pd.resize_img(img, (416, 416))
fig, ax = plt.subplots()
ax.imshow(resize_img)
visual.visualize_joints_2d(ax, uvd_gt_resize_i[REORDER], joint_idxs=False)

In [ ]:
uvd_gt_upsize = pd.scale_annot_wh(uvd_gt_resize_i, (416, 416), (1920, 1080))
fig, ax = plt.subplots()
ax.imshow(img)
visual.visualize_joints_2d(ax, uvd_gt_upsize[REORDER], joint_idxs=False)

In [ ]:
ccs_gt_resize_i = xyzuvd.uvd2ccs_color(uvd_gt_resize_i)
uvd_new = xyzuvd.ccs2uvd_color(ccs_gt_resize_i)
fig, ax = plt.subplots()
ax.imshow(resize_img)
visual.visualize_joints_2d(ax, uvd_new[REORDER], joint_idxs=False)

In [ ]:
uvd_new_new = uvd_new.copy()
# uvd_new_new[..., 0] = (uvd_new[..., 0] - np.amin(uvd_new[..., 0])) /(np.amax(uvd_new[..., 0] - np.amin(uvd_new[..., 0])))
# uvd_new_new[..., 1] = (uvd_new[..., 1] - np.amin(uvd_new[..., 1])) /(np.amax(uvd_new[..., 1] - np.amin(uvd_new[..., 1])))
# uvd_new_new[..., 2] = (uvd_new[..., 2] - np.amin(uvd_new[..., 2])) /(np.amax(uvd_new[..., 2] - np.amin(uvd_new[..., 2])))

# root_pos = uvd_new_new[0]

# uvd_new_new = uvd_new.copy()
# uvd_new_new = (uvd_new - (uvd_new[0] - root_pos)) 

uvd_new_new[:, 0] -= (uvd_new[0, 0]//32)*32
uvd_new_new[:, 1] -= (uvd_new[0, 1]//32)*32

import matplotlib.patches as patches
fig, ax = plt.subplots(figsize=(13,13))
ax.imshow(resize_img)
visual.visualize_joints_2d(ax, uvd_new_new[REORDER], joint_idxs=False)
for i in range(169):
    index = np.unravel_index(i, (13, 13))
    x, y = index
    rect = patches.Rectangle((9*32,12*32),32,32,linewidth=1,edgecolor='b', facecolor='b', fill=True, alpha=0.5)
    ax.add_patch(rect)